# Imports

In [ ]:
import os

import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import neptune
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
load_dotenv()

# device = "cuda"
device = "mps"

# Dataset

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    target_transform=transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
    target_transform=transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# Model

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3),
            nn.MaxPool2d(2,stride=1),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size=3),
            nn.MaxPool2d(2,stride=1),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3),
            nn.MaxPool2d(2,stride=2),
            nn.ReLU()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(6400, 256),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, 10),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 6400)
        x = self.fc1(x)
        logits = self.fc2(x)
        return logits

# Experiment

## Loops

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = []
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            losses.append(loss.item())
    return losses


def test_loop(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    losses, accuracy = [], []

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X.to(device))
            test_loss = loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.argmax(1).to(device)).type(torch.float).sum().item()

            if batch % 50 == 49:
                losses.append(test_loss)
                accuracy.append(correct / (pred.shape[0]*100))
                correct = 0
    return losses, accuracy

In [ ]:
def experiment(train_dataloader, test_dataloader, model, loss_fn, optimizer, run, epochs = 100):
    train_losses, test_losses, test_accuracy = [], [], []
    for epoch in tqdm(range(epochs), desc="Epoch"):
        train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
        train_loss = np.mean(train_loss)
        train_losses.append(train_loss)
        run["train/loss"].append(train_loss)

        test_loss, test_acc = test_loop(test_dataloader, model, loss_fn)
        test_loss = np.mean(test_loss)
        test_losses.append(test_loss)
        run["test/loss"].append(test_loss)
        test_accuracy += test_acc
        run["test/acc"].append(np.mean(test_acc))
    return train_losses, test_losses, test_accuracy

In [ ]:
def init_neptune_run():
    return neptune.init_run(project=os.getenv("NEPTUNE_PROJECT"), 
                            api_token=os.getenv("NEPTUNE_API_TOKEN"), 
                            capture_stdout=False, 
                            capture_stderr=False, 
                            capture_traceback=False, 
                            capture_hardware_metrics=False)

In [ ]:
def plot_results(train_losses, test_losses, test_accuracy, name):
    fig, axs = plt.subplots(1,2, figsize=(11,5))
    axs[0].plot(train_losses, label= "train")
    axs[0].plot(test_losses, label="test")
    axs[0].legend()
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].set_title("Training and test losses")

    axs[1].plot(test_accuracy)
    axs[1].set_xlabel("3 values per epoch (every 50th batch)")
    axs[1].set_ylabel("Accuracy")
    axs[1].set_title("Test accuracy")
    plt.suptitle(name)
    plt.show()

## Runs

In [ ]:
model = ConvolutionalNeuralNetwork().to(device)
run = init_neptune_run()
run["parameters"] = {"optimizer": "SGD"}
train_losses, test_losses, test_accuracy = experiment(train_dataloader, test_dataloader,
                                                      model, nn.CrossEntropyLoss().to(device),
                                                      torch.optim.SGD(model.parameters(), lr=1e-3),
                                                      run)
run.stop()
plot_results(train_losses, test_losses, test_accuracy, "SGD")

In [ ]:
model = ConvolutionalNeuralNetwork().to(device)
run = init_neptune_run()
run["parameters"] = {"optimizer": "Adagrad"}
train_losses, test_losses, test_accuracy = experiment(train_dataloader, test_dataloader,
                                                      model, nn.CrossEntropyLoss().to(device),
                                                      torch.optim.Adagrad(model.parameters(), lr=1e-5),
                                                      run)
run.stop()
plot_results(train_losses, test_losses, test_accuracy, "Adagrad")

In [ ]:
model = ConvolutionalNeuralNetwork().to(device)
run = init_neptune_run()
run["parameters"] = {"optimizer": "Adam"}
train_losses, test_losses, test_accuracy = experiment(train_dataloader, test_dataloader,
                                                      model, nn.CrossEntropyLoss().to(device),
                                                      torch.optim.Adam(model.parameters(), lr=1e-5),
                                                      run)
run.stop()
plot_results(train_losses, test_losses, test_accuracy, "Adam")